In [4]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', 'LM(다국어)', '번역(다국어-한)', '번역(한-다국어)', '한국어 번역 교정', '다국어 번역 교정']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
text_set = set()
korean = 'ko-KR'

#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            source_language = line['S_Code']
            source_text = line['원문'].replace('  ', ' ').strip()
            target_language = line['T_Code']
            preference_order = ['MT', '1차수정', '2차수정', '최종번역문']
            target_texts = ['']
            for pref in preference_order:
                text = str(line[pref]).replace('  ', ' ').strip()
                if text and target_texts[-1] != text:
                    target_texts.append(text)
            target_texts = list(reversed(target_texts))[:-1]
            #### data preprocess end 
            
            #### LM
            
            if sum(ko_id:=[source_language==korean, target_language==korean]):
                data = {'text': None}
                if ko_id[0]:
                    text_korean = source_text
                    text_multi = target_texts[0]
                    lang_multi = target_language
                elif ko_id[1]:
                    text_multi= source_text
                    text_korean = target_texts[0]
                    lang_multi = source_language
                
                if text_korean not in text_set:              
                    text_set.add(text_korean)
                    data['text'] = text_korean
                    task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                    # print(json.dumps(data, indent=4, ensure_ascii=False))
                    
                if text_multi not in text_set:              
                    text_set.add(text_multi)
                    data['text'] = {'language':lang_multi, 'text' : text_multi}
                    task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                    # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM end
            
            #### 번역
            data = {'input': {}, 'output': None}
            data['input']['source_text'] = source_text
            data['input']['source_language'] = source_language
            data['input']['target_language'] = target_language
            data['output'] = target_texts[0]
            
            #### 번역(다국어-한)
            if target_language == 'ko-KR':
                task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(다국어-한) end
            
            
            #### 번역(한-다국어)
            if source_language == 'ko-KR':
                task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(한-다국어) end

            if len(target_texts) > 1:
                data['output'] = target_texts

                if target_language == 'ko-KR':
                    #### 한국어 번역 교정
                    ### 'output' is list of output sorted by relevance (1st is most relevant)
                    task_files[4].write(json.dumps(data, ensure_ascii=False)+'\n')
                    # print(json.dumps(data, indent=4, ensure_ascii=False))
                    #### 한국어 번역 교정 end
                    
                if source_language == 'ko-KR':
                    #### 다국어 번역 교정
                    ### 'output' is list of output sorted by relevance (1st is most relevant)
                    task_files[5].write(json.dumps(data, ensure_ascii=False)+'\n')
                    # print(json.dumps(data, indent=4, ensure_ascii=False))
                    #### 다국어 번역 교정 end
            
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 12/12 [00:06<00:00,  1.89it/s]
